In [ ]:
!nvidia-smi
!pip install -q transformers peft datasets accelerate bitsandbytes sentencepiece

Sun Nov 16 15:47:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, json, torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)

# -----------------------------
# 🔥 CHOOSE WHICH TASK TO TRAIN
# -----------------------------
# Options:
#  "resume_eval"
#  "latex_resume"
#  "job_match"
#  "recruiter_dialog"

TASK_NAME = "resume_eval"   # ⬅⬅⬅ change this for each adapter

# -----------------------------
# 🔥 Dataset paths
# -----------------------------
DATA_DIR = "/content/drive/MyDrive/models/datasets"
DATA_PATH = f"{DATA_DIR}/{TASK_NAME}.jsonl"

# -----------------------------
# 🔥 Phi-3 base model
# -----------------------------
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

# -----------------------------
# 🔥 Save directory
# -----------------------------
SAVE_DIR = f"/content/drive/MyDrive/models/qlora_adapters/{TASK_NAME}"
os.makedirs(SAVE_DIR, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 4-bit configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto",
)

model = prepare_model_for_kbit_training(model)
model.config.use_cache = False  # for gradient checkpointing compatibility

print("Model loaded in 4-bit.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model loaded in 4-bit.


In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 3,145,728 || all params: 3,824,225,280 || trainable%: 0.0823


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
import json
from datetime import date, datetime

# Converts ANY object to JSON-safe string
def safe_json(obj):
    try:
        return json.dumps(obj, indent=2, default=str)
    except:
        return str(obj)

def format_example(example):
    instruction = example["instruction"]
    input_data = example["input"]
    output_data = example["output"]

    # Convert everything to JSON-safe strings
    pretty_input = safe_json(input_data)

    # Output may be dict, LaTeX, list, dialogue, etc.
    if isinstance(output_data, dict) or isinstance(output_data, list):
        pretty_output = safe_json(output_data)
    else:
        pretty_output = str(output_data)

    text = (
        "<|system|>\nYou are a helpful AI assistant.\n"
        "<|user|>\n"
        f"Instruction:\n{instruction}\n\n"
        f"Input JSON:\n{pretty_input}\n\n"
        "<|assistant|>\n"
        f"{pretty_output}"
    )

    return {"text": text}

# Apply formatting safely
raw_dataset = load_dataset("json", data_files=DATA_PATH)["train"]
formatted_dataset = raw_dataset.map(format_example)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
MAX_LENGTH = 1024  # enough for resumes + LaTeX + ranking

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
    )

tokenized_dataset = formatted_dataset.map(
    tokenize,
    batched=True,
    remove_columns=formatted_dataset.column_names
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

len(tokenized_dataset)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

200

In [ ]:
training_args = TrainingArguments(
    output_dir=SAVE_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=2,           # you can increase to 2 if dataset is tiny
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    warmup_steps=10,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

TypeError: Phi3ForCausalLM.forward() got an unexpected keyword argument 'num_items_in_batch'

In [ ]:
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print(f"Adapter saved to: {SAVE_DIR}")


Adapter saved to: /content/drive/MyDrive/models/qlora_adapters/resume_eval


In [ ]:
from huggingface_hub import snapshot_download
import os

TARGET_DIR = "/content/drive/MyDrive/models/phi3_base"

# Create the folder manually
os.makedirs(TARGET_DIR, exist_ok=True)

snapshot_download(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    local_dir=TARGET_DIR,
    local_dir_use_symlinks=False,
    ignore_patterns=["*.msgpack"]  # avoids cache-only files
)

print("Download complete: ", TARGET_DIR)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

SECURITY.md: 0.00B [00:00, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

CODE_OF_CONDUCT.md:   0%|          | 0.00/453 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

NOTICE.md: 0.00B [00:00, ?B/s]

sample_finetune.py: 0.00B [00:00, ?B/s]

Download complete:  /content/drive/MyDrive/models/phi3_base
